In [1]:
import numpy as np
import math
%matplotlib qt
import matplotlib.pyplot as plt

from scipy import ndimage as ndi
import pandas as pd

from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import


from skimage.morphology import watershed
from skimage.feature import peak_local_max
from PIL import Image

from skimage.morphology import skeletonize, thin, skeletonize_3d
from skimage.util import invert
from skimage import measure
from skimage.morphology import erosion, dilation, opening, closing, white_tophat
from skimage.morphology import black_tophat, skeletonize, convex_hull_image
from skimage.morphology import disk
from skimage import feature
from skimage.filters import roberts, sobel, scharr, prewitt
import seaborn as sns
sns.reset_orig()

from skimage import restoration

from skimage import exposure
from skimage.filters import gaussian

from scipy import ndimage

from time import time

from skimage.color import rgb2gray


from skimage.filters.rank import entropy
import scipy



import networkx as nx
from scipy import interpolate

from skimage.segmentation import random_walker
from skimage.data import binary_blobs
from skimage.exposure import rescale_intensity
import glob

from skimage.morphology import disk
from skimage import io
from skimage.morphology import remove_small_objects

import threading

import csv

In [2]:
def get_centroids(arr):
    
    arr= arr!=0
    markers, nummarkers = ndi.label(arr) 
    
    points = np.where(arr)
    points = np.asarray(points)
    
    pt_labels = markers[points[0], points[1], points[2]]
    
    Z_mass = (np.bincount(pt_labels, weights=points[0]))[1:]

    Y_mass = (np.bincount(pt_labels, weights=points[1]))[1:]

    X_mass = (np.bincount(pt_labels, weights=points[2]))[1:]

    label_occurances = (np.bincount(pt_labels, weights=np.ones((points.shape[1]))) )[1:]
    
    
    Z_centroid = Z_mass/label_occurances
    Y_centroid = Y_mass/label_occurances
    X_centroid = X_mass/label_occurances
    
    return Z_centroid, Y_centroid, X_centroid, label_occurances

In [3]:
# each pixel = number of times the labled occurs
def get_occurances(arr):
    
    arr= arr!=0
    markers, nummarkers = ndi.label(arr) 
    
    points = np.where(arr)
    points = np.asarray(points)
    
    pt_labels = markers[points[0], points[1], points[2]]

    label_occurances = (np.bincount(pt_labels, weights=np.ones((points.shape[1]))) )[1:]
    
    
    return label_occurances

In [4]:
# 
def get_endpoints_occurrances(arr, epts):
    arr= arr!=0
    markers, nummarkers = ndi.label(arr) 
    
    arr = arr*(epts!=0)
    
    points = np.where(arr)
    points = np.asarray(points)
    
    pt_labels = markers[points[0], points[1], points[2]]

    label_occurances = (np.bincount(pt_labels, weights=np.ones((points.shape[1]))) )[1:]
    
    
    return label_occurances
    

In [5]:
def pair_labels_W_vals(arr, labels, vals):
    arr= arr!=0
    markers, nummarkers = ndi.label(arr) 
    
    points = np.where(arr)
    points = np.asarray(points)
    
    pt_labels = markers[points[0], points[1], points[2]]
    
    dict_1 = dict(zip(labels, vals))
    
    new_labels = np.vectorize(dict_1.get)(pt_labels)
    
    markers[points[0], points[1], points[2]] = new_labels
    
    return markers

In [5]:
def getNeighbors(arr):
    
    arr2 = (arr.copy())*0
    
    
    #IN YZ PLANE
    
    #bottom front
    arr2[:-1,:-1] += arr[1:,1:]
    
    
    
    #bottom 
    arr2[:-1] += arr[1:]
    
    
    #bottom back
    arr2[:-1, 1:] += arr[1:, :-1]
    
    #top front
    arr2[1:, :-1] += arr[:-1, 1:]
    #top
    arr2[1:] += arr[:-1]
    #top back
    arr2[1:, 1:] += arr[:-1, :-1]
    
    #front
    arr2[:, :-1] += arr[:, 1:]
    #back
    arr2[:, 1:] += arr[:, :-1]
    
    
    
    #LEFT LEFT LEFT LEFT LEFT LEFT LEFT 
    #LEFT LEFT LEFT LEFT LEFT LEFT LEFT
    #LEFT LEFT LEFT LEFT LEFT LEFT LEFT
    
    #bottom front
    arr2[:-1,:-1, 1:] += arr[1:,1:, :-1]
    #bottom 
    arr2[:-1, :, 1:] += arr[1:, :, :-1]
    #bottom back
    arr2[:-1, 1:, 1:] += arr[1:, :-1, :-1]
    
    #top front
    arr2[1:, :-1, 1:] += arr[:-1, 1:, :-1]
    #top
    arr2[1:, :, 1:] += arr[:-1, :, :-1]
    #top back
    arr2[1:, 1:, 1:] += arr[:-1, :-1, :-1]
    
    #front
    arr2[:, :-1, 1:] += arr[:, 1:, :-1]
    #back
    arr2[:, 1:, 1:] += arr[:, :-1, :-1]
    
    #just left 
    arr2[:, :, 1:] += arr[:, :, :-1]
    
    
    
    #RIGHT RIGHT RIGHT RIGHT RIGHT RIGHT
    #RIGHT RIGHT RIGHT RIGHT RIGHT RIGHT
    #RIGHT RIGHT RIGHT RIGHT RIGHT RIGHT
    
    #bottom front
    arr2[:-1,:-1, :-1] += arr[1:,1:, 1:]
    #bottom 
    arr2[:-1, :, :-1] += arr[1:, :, 1:]
    #bottom back
    arr2[:-1, 1:, :-1] += arr[1:, :-1, 1:]
    
    #top front
    arr2[1:, :-1, :-1] += arr[:-1, 1:, 1:]
    #top
    arr2[1:, :, :-1] += arr[:-1, :, 1:]
    #top back
    arr2[1:, 1:, :-1] += arr[:-1, :-1, 1:]
    
    #front
    arr2[:, :-1, :-1] += arr[:, 1:, 1:]
    #back
    arr2[:, 1:, :-1] += arr[:, :-1, 1:]
    
    
    #just right 
    arr2[:, :, :-1] += arr[:, :, 1:]
    
    
    
    return arr2*(arr!=0)

In [6]:
# Mac
# segs = io.imread("/Users/spencerlab/Desktop/Temporary_Computational_Folders/Blood_Vessels/Graph_development/skel_Segs_3D.tif")

# Desktop 1
segs = io.imread("/Users/Spenc/OneDrive/Desktop/Kumaran/bv/skel_Segs_3D.tif")

segs = segs*(getNeighbors(segs!=0)>0) # gets rid of all single pixel segments


In [7]:
# Mac
# centroid_csv = pd.read_csv("/Users/spencerlab/Desktop/Temporary_Computational_Folders/Blood_Vessels/Graph_development/Day3_mouse1_Nodes.csv") 

# Desktop 2
centroid_csv = pd.read_csv("/Users/Spenc/OneDrive/Desktop/Kumaran/bv/Day3_mouse1_Nodes.csv")

centroids = centroid_csv.to_numpy()

In [8]:
node_num = 102709

# TODO: need to make sure that centroid has space above/below
Center_Z = int(centroids[node_num][1])
Center_Y = int(centroids[node_num][2])
Center_X = int(centroids[node_num][3])

#  size of substack (each side)
Z_margin = 15
Y_margin = 15
X_margin = 15

substack = segs[Center_Z-Z_margin:Center_Z+Z_margin+1,Center_Y-Y_margin:Center_Y+Y_margin+1, Center_X-X_margin:Center_X+X_margin+1]

In [10]:
substack.shape

(31, 31, 31)

In [ ]:
# create (z)(y)(x) arrays of substack
# label the substack
# for each label - get endpoints (a, b)
# follow endpoint a --> b, add locations to list
# create parameterized line for each segment

In [9]:
labeled_substack, num_segs = ndi.measurements.label(substack, np.ones((3,3,3)))
enpt_substack = getNeighbors(substack!=0) == 1

In [12]:
num_segs

42

In [13]:
label = 4

z_par = []
y_par = []
x_par = []

loc_par = [] # stores z,y,x as tuple, makes it easy to compare
    
seg = labeled_substack == label # essentially the stack with only that segment
label_loc = np.where(seg) # list of locations of each point in segment
    
 # get endpoints - TODO:  replace w kumaran's function
endpt = enpt_substack*(seg) 
endpt_loc = np.where(endpt)

In [14]:
endpt_loc

(array([0, 1], dtype=int64),
 array([29, 25], dtype=int64),
 array([21, 27], dtype=int64))

In [15]:
label_loc

(array([0, 0, 0, 0, 0, 1, 1], dtype=int64),
 array([27, 28, 29, 29, 29, 25, 26], dtype=int64),
 array([25, 24, 21, 22, 23, 27, 26], dtype=int64))

In [16]:
# whatever endpoint is first in np.where
cp_z = endpt_loc[0][0]
cp_y = endpt_loc[1][0]
cp_x = endpt_loc[2][0]

In [29]:
def checkNeighbor(diff_z, diff_y, diff_x):
    distance_sq = diff_z+diff_y+diff_x
    
    if ((distance_sq == 1) or (distance_sq == 2) or (distance_sq == 3)):
        return True;

In [30]:
z_par = []
y_par = []
x_par = []

loc_par = [] # stores z,y,x as tuple, makes it easy to compare

loc_par.append((cp_z,cp_y,cp_x))
z_par.append(cp_z)
y_par.append(cp_y)
x_par.append(cp_x)

for i in range(len(label_loc[0])): # iterates through each label location
    
        print(i)
        z = (label_loc[0][i])
        y = (label_loc[1][i])
        x = (label_loc[2][i])


        # to be a neighbor, the difference has to be either +/- 1
        diff_z_sq = (cp_z - z)**4
        diff_y_sq = (cp_y - y)**4
        diff_x_sq = (cp_x - x)**4
        
        # if its a neighbor, 
        if (checkNeighbor(diff_z_sq, diff_y_sq, diff_x_sq) and ((z,y,x) not in loc_par)):
            loc_par.append((z,y,x))
            z_par.append(z)
            y_par.append(y)
            x_par.append(x)
            
            cp_z = z
            cp_y = y
            cp_x = x
            
            '''for j in range(len(label_loc[0])):
                z = (label_loc[0][j])
                y = (label_loc[1][j])
                x = (label_loc[2][j])
                
                diff_z_sq = (cp_z - z)**4
                diff_y_sq = (cp_y - y)**4
                diff_x_sq = (cp_x - x)**4
                
                
                if (checkNeighbor(diff_z_sq, diff_y_sq, diff_x_sq) and ((z,y,x) not in loc_par)):
                
                    loc_par.append((z,y,x))
                    z_par.append(z)
                    y_par.append(y)
                    x_par.append(x)
                    
                    cp_z = z
                    cp_y = y
                    cp_x = x'''
 

0
1
2
3


In [24]:
plt.rcParams['legend.fontsize'] = 10

fig = plt.figure()
ax = fig.gca(projection='3d')

# Prepare arrays x, y, z
z = z_par
y = y_par
x = x_par

ax.plot(x, y, z, label="segment"+str(label))
ax.legend()

plt.show()

In [23]:
loc_par

[(0, 28, 24),
 (0, 27, 25),
 (0, 29, 21),
 (0, 29, 22),
 (0, 29, 23),
 (1, 25, 27),
 (1, 26, 26)]

In [ ]:
math.sqrt(1+1)

In [ ]:
######## FINAL ########

In [12]:
plt.rcParams['legend.fontsize'] = 10

fig = plt.figure()
ax = fig.gca(projection='3d')

for each_label in range(1, num_segs): # each label should be a separate segment
    
    z_par = []
    y_par = []
    x_par = []

    loc_par = [] # stores z,y,x as tuple, makes it easy to compare
    
    seg = labeled_substack == each_label # essentially the stack with only that segment
    label_loc = np.where(seg) # list of locations of each point in segment
    
    # get endpoints
    endpt = enpt_substack*(seg) 
    endpt_loc = np.where(endpt)
    
    
    # whatever endpoint is first in np.where
    cp_z = endpt_loc[0][0]
    cp_y = endpt_loc[1][0]
    cp_x = endpt_loc[2][0]

    loc_par.append((cp_z,cp_y,cp_x))
    z_par.append(cp_z)
    y_par.append(cp_y)
    x_par.append(cp_x)

    for i in range(len(label_loc[0])): # iterates through each label location
        z = (label_loc[0][i])
        y = (label_loc[1][i])
        x = (label_loc[2][i])


        # to be a neighbor, the difference has to be either +/- 1
        diff_z_sq = (cp_z - z)**2
        diff_y_sq = (cp_y - y)**2
        diff_x_sq = (cp_x - x)**2
        
        distance = math.sqrt(diff_z_sq+diff_y_sq+diff_x_sq)
        # distance has to be 1, sqrt(2), sqrt(3)
        
        # if its a neighbor, 
        if ((distance == 1) or (distance == 1.7320508075688772) or (distance == 1.4142135623730951)) and (((z,y,x) not in loc_par)):
            loc_par.append((z,y,x))
            z_par.append(z)
            y_par.append(y)
            x_par.append(x)
            
            cp_z = z
            cp_y = y
            cp_x = x
            
            for j in range(len(label_loc[0])):
                z = (label_loc[0][j])
                y = (label_loc[1][j])
                x = (label_loc[2][j])
                
                if ((distance == 1) or (distance == 1.7320508075688772) or (distance == 1.4142135623730951)) and (((z,y,x) not in loc_par)):
                
                    loc_par.append((z,y,x))
                    z_par.append(z)
                    y_par.append(y)
                    x_par.append(x)
                    
                    cp_z = z
                    cp_y = y
                    cp_x = x
    
    ax.plot(x_par, y_par, z_par, label="Segment "+str(each_label))
    ax.legend()
    
    
    

In [ ]:

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(x, y, z, c='r', marker='o')

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

plt.show()